# Setup

## Setting up Output Format using Pydantic

In [1]:
from pydantic import BaseModel, Field
from typing import List, Optional

class Experience(BaseModel):
    title: str = Field(description="Job title")
    organization: str = Field(description="Organization name")
    start_date: str = Field(description="Start date")
    end_date: Optional[str] = Field(description="End date")
    description: str = Field(description="Job description")

class Education(BaseModel):
    credential_name: str = Field(description="Credential name")
    institution_name: str = Field(description="Institution name")
    start_date: str = Field(description="Start date")
    end_date: str = Field(description="End date")
    description: str = Field(description="Education description")

class ParsedResume(BaseModel):
#     github_url: str = Field(description="Github Profile URL")
#     linkedin_url: str = Field(description="Linkedin Profile URL")
    experiences: List[Experience] = Field(description="List of experiences")
#     education_history: List[Education] = Field(description="List of education history")

In [2]:
from langchain.output_parsers import PydanticOutputParser

resume_output_parser = PydanticOutputParser(pydantic_object=ParsedResume)

## Set up Prompt

In [3]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template="I am providing a JSON Schema followed by a resume in plain text. Format the plain text into the JSON format.\n{format_instructions}\n{resume}\n",
    input_variables=["resume"],
    partial_variables={"format_instructions": resume_output_parser.get_format_instructions()},
)

## Large-Language Model (LLM)

In [4]:
from langchain.llms import OpenAI

model_name = "text-davinci-003"
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature)

# Trying it out

## Taking resume input

In [5]:
with open('test_data/fictional-resume.md') as input_file:
    resume_content = input_file.read()

In [6]:
print(resume_content)

**Jonathon Bracken**

1234 Silicon Valley Rd., San Jose, CA 95126
(408) 123-4567 | jonathon.bracken@example.com
LinkedIn: linkedin.com/in/jonathon-bracken

**Objective**
A versatile Software Engineer with over 10 years of diverse experience in front-end, back-end, and Android firmware engineering roles. Eager to apply problem-solving abilities and technical expertise in a challenging new role.

---

**Skills**

- Programming languages: Java, JavaScript, Python, C++, C#, Kotlin, Swift, SQL
- Web Technologies: HTML, CSS, React, AngularJS, Node.js, Express.js, REST APIs
- Databases: MySQL, PostgreSQL, MongoDB, Redis
- Android Firmware: Android Open Source Project (AOSP), Linux Kernel, Custom ROM Development
- Tools: Git, Docker, Jenkins, Jira, Agile/Scrum methodologies
- Soft Skills: Communication, Leadership, Problem Solving, Teamwork, Adaptability

---

**Work Experience**

**Senior Software Engineer | HoloWare Inc.**
*San Jose, CA | February 2021 - Present*

- Lead the development team

In [7]:
_input = prompt.format_prompt(resume=resume_content)

In [8]:
output = model(_input.to_string())

In [9]:
from langchain.output_parsers import OutputFixingParser
from langchain.chat_models import ChatOpenAI

output_fixing_parser = OutputFixingParser.from_llm(parser=resume_output_parser, llm=ChatOpenAI())

In [10]:
resume = output_fixing_parser.parse(output)

In [11]:
import json
json_formatted_str = json.dumps(resume.dict(), indent=2)
print(json_formatted_str)

{
  "experiences": [
    {
      "title": "Senior Software Engineer",
      "organization": "HoloWare Inc.",
      "start_date": "February 2021",
      "end_date": "Present",
      "description": "Lead the development team of HoloLens, a holographic glasses project, developing firmware based on Android Open Source Project (AOSP). Enhanced system performance by optimizing Linux Kernel and effectively reduced boot-up time by 30%. Assisted in the development of several in-house tools to facilitate rapid firmware testing and deployment."
    },
    {
      "title": "Full Stack Developer",
      "organization": "Nebula Dynamics",
      "start_date": "June 2017",
      "end_date": "February 2021",
      "description": "Contributed to a team responsible for developing Nebula Cloud Suite, a comprehensive suite of cloud-based tools for businesses. Developed RESTful APIs using Node.js and Express, which served over 1M requests per day with an average response time of 200ms. Created responsive an